In [1]:
%autosave 0

Autosave disabled


In [2]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

In [3]:
import grpc
import tensorflow as tf
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

2024-12-16 11:10:50.122635: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-16 11:10:50.474889: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-12-16 11:10:50.475019: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-12-16 11:10:52.242054: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

In [4]:
print(tf.__file__)

/home/mike/developer/ml-engineer-zoomcamp/.venv/lib/python3.10/site-packages/tensorflow/__init__.py


In [5]:
host = 'localhost:8500'

channel = grpc.insecure_channel(host)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [6]:
from keras_image_helper import create_preprocessor


In [7]:
preprocessor = create_preprocessor('xception', target_size=(299, 299))


In [8]:
url = 'http://bit.ly/mlbookcamp-pants'
X = preprocessor.from_url(url)

In [9]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [12]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = 'clothing-model'
pb_request.model_spec.signature_name = 'serving_default'

pb_request.inputs['input_8'].CopyFrom(np_to_protobuf(X))

In [13]:
pb_response = stub.Predict(pb_request, timeout=20.0)

In [16]:
preds = pb_response.outputs['dense_7'].float_val

In [17]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

In [18]:
dict(zip(classes, preds))

{'dress': -1.8798640966415405,
 'hat': -4.75631046295166,
 'longsleeve': -2.359531879425049,
 'outwear': -1.0892632007598877,
 'pants': 9.90378189086914,
 'shirt': -2.8261773586273193,
 'shoes': -3.6483097076416016,
 'shorts': 3.241151809692383,
 'skirt': -2.6120948791503906,
 't-shirt': -4.852035999298096}